In [43]:
using LinearAlgebra
using InvertedIndices
Y=[1/0.0576im 0.0+0.0im 0.0+0.0im -1/0.0576im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im;0.0+0.0im 1/(0.0625im) 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0625im) 0.0+0.0im;0.0+0.0im 0.0+0.0im 1/0.0586im 0.0+0.0im 0.0+0.0im -1/0.0586im 0.0+0.0im 0.0+0.0im 0.0+0.0im;-1/0.0576im 0.0+0.0im 0.0+0.0im 1/0.0576im+1/(0.0170+0.0920im)+1/(0.0100+0.0850im) -1/(0.0170+0.0920im) 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0100+0.0850im);0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0170+0.0920im) 1/(0.0170+0.0920im)+1/(0.0390+0.1700im) -1/(0.0390+0.1700im) 0.0+0.0im 0.0+0.0im 0.0+0.0im;0.0+0.0im 0.0+0.0im -1/0.0586im 0.0+0.0im -1/(0.0390+0.1700im) 1/(0.0390+0.1700im)+1/0.0586im+1/(0.0119+0.1008im) -1/(0.0119+0.1008im) 0.0+0.0im 0.0+0.0im;0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0119+0.1008im) 1/(0.0119+0.1008im)+1/(0.0085+0.0720im) -1/(0.0085+0.0720im) 0.0+0.0im;0.0+0.0im -1/(0.0625im) 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0085+0.0720im) 1/(0.0085+0.0720im)+1/(0.0625im)+1/(0.032+0.1610im) -1/(0.0320+0.1610im);0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0100+0.0850im) 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0320+0.1610im) 1/(0.0320+0.1610im)+1/(0.0100+0.0850im)]
S = [0.723+0.2703im,1.63+0.0654im,0.85-0.1095im,0.0+0.0im,-0.9-0.3im,0.0+0.0im,-1.0-0.35im,0.0+0.0im,-1.25-0.5im]
Vm = [1.04,1.025,1.025]
V = [1.04+0.0im,1.025+0.0im,1.025+0.0im,1.0+0.0im,1.0+0.0im,1.0+0.0im,1.0+0.0im,1.0+0.0im,1.0+0.0im]
PQ = [4,5,6,7,8,9]
Swing=[1]
PV = [2,3]
N = 9
ΔV =0.0
θ = 0.0
#======================================================Function Declaration==================================================#
function sudut(V)
    # menghitung sudut bilangan kompleks dalam radian
    x=real(V)
    y=imag(V)
    θ=atan.(y./x)
    return θ
end
function radiankederajat(rad)
    hasil = rad.*180/pi
    return hasil
end
function Current(N,Y,V) #menghitung arus berdasarkan pers 11
        In = zeros(ComplexF64,N,1)
        for k in 1:N
            for n in 1:N
                In[k] = In[k] + Y[k,n]*V[n]
            end
        end
        return In
        end
function Missmatch(i,V,S,PV,PQ,N) # missmatch berdasarkan pers 10 dan 24
        f = zeros(Float64,2*N,1)
        for k in PV                      #bus PV
            f[2*k-1] = real(conj(V[k])*i[k]) - real(S[k])
            f[2*k] = abs(V[k])^2 - Vm[k]^2
        end
        for k in PQ                     #bus PQ
            f[2*k-1] = real(conj(V[k])*i[k]- S[k])
            f[2*k] = imag(conj(V[k])*i[k] + S[k])
        end
        f = f[Not(1:2)]
        return f
        end

function Jacobian(V,i,Y,N,PQ,PV)
        J = zeros(Float64,2*N,2*N)
    for k in PV               #bus PV perhitungan jacobian berdasarkan pers  28,29,30,31
            for n in 1:N
                if k!=n
                #off diagonal
                    J[2*k-1,n] = real(conj(V[k])*Y[k,n])
                    J[2*k,n] = 0                                #terhadap V real
                    J[2*k-1,N+n] =real(conj(V[k])*1im*Y[k,n])
                    J[2*k,N+n] = 0                               #terhadap V imag
                else
                #diagonal
                    J[2*k-1,n] = real(i[k]+conj(V[k])*Y[k,n])
                    J[2*k,n] = 2*real(V[k])                       #terhadap V real
                    J[2*k-1,N+n] =real(-1im*i[k]+conj(V[k])*1im*Y[k,n])
                    J[2*k,N+n] = 2*imag(V[k])                      #terhadap V imag
                end
            end
        end
        for k in PQ                #bus PQ perhitungan jacobian berdasarkan pers 14,15,18,19
            for n in 1:N
                if k!=n
                #off diagonal
                    J[2*k-1,n] = real(conj(V[k])*Y[k,n])
                    J[2*k,n] = imag(conj(V[k])*Y[k,n])          #terhadap V real
                    J[2*k-1,N+n] =real(conj(V[k])*1im*Y[k,n])
                    J[2*k,N+n] = imag(conj(V[k])*1im*Y[k,n])    #terhadap V imag
                else
                #diagonal
                    J[2*k-1,n] = real(i[k]+conj(V[k])*Y[k,n])
                    J[2*k,n] = imag(i[k]+conj(V[k])*Y[k,n])     #terhadap V real
                    J[2*k-1,N+n] =real(-1im*i[k]+conj(V[k])*1im*Y[k,n])
                    J[2*k,N+n] = imag(-1im*i[k]+conj(V[k])*1im*Y[k,n])  ##terhadap V imag
                end
            end
        end
        J = J[Not(1:2), Not(1,N+1)] # hasil jacobian dengan nilai bus 1 tidak ditampilkan
        return J
        end
function S_cal(i,V,PV,PQ,N)       #nilai daya
        Scal = zeros(Float64,2*N,1)
    for k in PV
            Scal[2*k-1] = real(conj(V)[k]*i[k])
            Scal[2*k] = imag(conj(V)[k]*i[k])
        end
        for k in PQ
            Scal[2*k-1] = real(conj(V)[k]*i[k])
            Scal[2*k] = imag(conj(V)[k]*i[k])
        end
        Scal = Scal[Not(1:2)]
        return Scal
        end
function update_teg(V,Δv)
    ΔV = zeros(ComplexF64,N,1)
     for k in PV
    ΔV[k] = Δv[2*k-(k+1)]+Δv[7+k]*1im
    end
    for k in PQ
    ΔV[k] = Δv[2*k-(k+1)]+Δv[7+k]*1im
    end
V = V+ΔV
    return V
end
#####################################################iterasi##############################################################
H = 1
jumlah_iterasi = 0
 while H > 5e-5
 i = Current(N,Y,V)
 s = S_cal(i,V,PV,PQ,N)
 f = Missmatch(i,V,S,PV,PQ,N)
global H = maximum(abs.(f))
 j = Jacobian(V,i,Y,N,PQ,PV)
 Δv = -inv(j)*f 
 V = update_teg(V,Δv)
 θ = radiankederajat(sudut(V))
 global jumlah_iterasi += 1
  @time println(jumlah_iterasi)
   end

1
  0.000784 seconds (26 allocations: 672 bytes)
2
  0.002471 seconds (137 allocations: 10.016 KiB)
3
  0.000357 seconds (24 allocations: 800 bytes)
4
  0.002881 seconds (24 allocations: 896 bytes)


In [12]:
H = maximum(abs.(f))
jumlah_iterasi = 0
while true H > 5e-5
 i = Current(N,Y,V)
 s = S_cal(i,V,PV,PQ,N)
 f = Missmatch(i,V,S,PV,PQ,N)
 j = Jacobian(V,i,Y,N,PQ,PV)
 Δv = -inv(j)*f 
 V = update_teg(V,Δv)
 θ = radiankederajat(sudut(V))
 global jumlah_iterasi += 1
   println(H > 5e-5)
   end

  0.000000 seconds
elapsed time (ns): 0


9×1 Matrix{ComplexF64}:
     0.6955835919649993 - 0.7823798792282375im
     1.6552755129106365 - 0.24591358727973045im
     0.8582664427900385 - 0.30421363413294955im
  6.661338147750939e-16 - 1.7763568394002505e-15im
    -0.9092548505267328 + 0.37054479647222927im
  2.220446049250313e-16 + 3.552713678800501e-15im
    -1.0264004351967992 + 0.34250670677898576im
 -4.440892098500626e-16 - 8.881784197001252e-16im
    -1.2734702619421419 + 0.6194555973896989im

In [13]:
@timev V

  0.000000 seconds
elapsed time (ns): 342


9×1 Matrix{ComplexF64}:
               1.04 + 0.0im
 1.0103682203025721 + 0.17257189632907513im
 1.0212826217853517 + 0.0872170077406846im
 0.9949349189564536 - 0.04006561489718396im
   0.96415631680344 - 0.06297805961021574im
  1.003455702825161 + 0.036922594193188336im
 0.9790478879748424 + 0.014292211507102264im
  0.994998621097589 + 0.06911717677216035im
 0.9481981457219439 - 0.0686051739196467im

In [31]:
f

16-element Vector{Float64}:
  2.220446049250313e-16
  0.0
  0.0
  0.0
 -7.117082904738989e-17
  1.7673594480464273e-15
  0.0
  2.220446049250313e-16
  5.372603698338102e-16
 -3.589586190018618e-15
 -4.440892098500626e-16
 -3.3306690738754696e-16
 -5.401857317386992e-16
 -6.178480572126862e-15
  2.220446049250313e-16
 -1.6653345369377348e-16

In [20]:
f

16-element Vector{Float64}:
  2.220446049250313e-16
  0.0
  0.0
  0.0
 -7.117082904738989e-17
  1.7673594480464273e-15
  0.0
  2.220446049250313e-16
  5.372603698338102e-16
 -3.589586190018618e-15
 -4.440892098500626e-16
 -3.3306690738754696e-16
 -5.401857317386992e-16
 -6.178480572126862e-15
  2.220446049250313e-16
 -1.6653345369377348e-16

In [52]:
θ

9×1 Matrix{Float64}:
  0.0
  9.031101304366334
  4.430054961985804
 -2.2822218741295375
 -3.733358497855351
  1.7839527649621587
  0.5772962424221971
  3.577517022237991
 -4.1139986129002235

In [52]:
Y=[1/0.0576im 0.0+0.0im 0.0+0.0im -1/0.0576im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im;0.0+0.0im 1/(0.0625im) 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0625im) 0.0+0.0im;0.0+0.0im 0.0+0.0im 1/0.0586im 0.0+0.0im 0.0+0.0im -1/0.0586im 0.0+0.0im 0.0+0.0im 0.0+0.0im;-1/0.0576im 0.0+0.0im 0.0+0.0im 1/0.0576im+1/(0.0170+0.0920im)+1/(0.0100+0.0850im) -1/(0.0170+0.0920im) 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0100+0.0850im);0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0170+0.0920im) 1/(0.0170+0.0920im)+1/(0.0390+0.1700im) -1/(0.0390+0.1700im) 0.0+0.0im 0.0+0.0im 0.0+0.0im;0.0+0.0im 0.0+0.0im -1/0.0586im 0.0+0.0im -1/(0.0390+0.1700im) 1/(0.0390+0.1700im)+1/0.0586im+1/(0.0119+0.1008im) -1/(0.0119+0.1008im) 0.0+0.0im 0.0+0.0im;0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0119+0.1008im) 1/(0.0119+0.1008im)+1/(0.0085+0.0720im) -1/(0.0085+0.0720im) 0.0+0.0im;0.0+0.0im -1/(0.0625im) 0.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0085+0.0720im) 1/(0.0085+0.0720im)+1/(0.0625im)+1/(0.032+0.1610im) -1/(0.0320+0.1610im);0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0100+0.0850im) 0.0+0.0im 0.0+0.0im 0.0+0.0im -1/(0.0320+0.1610im) 1/(0.0320+0.1610im)+1/(0.0100+0.0850im)]


9×9 Matrix{ComplexF64}:
  0.0-17.3611im   0.0+0.0im    0.0+0.0im      …       0.0+0.0im
  0.0+0.0im       0.0-16.0im   0.0+0.0im              0.0+0.0im
  0.0+0.0im       0.0+0.0im    0.0-17.0648im          0.0+0.0im
 -0.0+17.3611im   0.0+0.0im    0.0+0.0im         -1.36519+11.6041im
  0.0+0.0im       0.0+0.0im    0.0+0.0im              0.0+0.0im
  0.0+0.0im       0.0+0.0im   -0.0+17.0648im  …       0.0+0.0im
  0.0+0.0im       0.0+0.0im    0.0+0.0im              0.0+0.0im
  0.0+0.0im      -0.0+16.0im   0.0+0.0im          -1.1876+5.97513im
  0.0+0.0im       0.0+0.0im    0.0+0.0im          2.55279-17.5792im

In [51]:
[0.0+0.0im 0.0+0.0im 1/0.0586im 0.0+0.0im 0.0+0.0im -1/0.0586im 0.0+0.0im 0.0+0.0im 0.0+0.0im]

1×9 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im  0.0-17.0648im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [12]:
@timed rand(10^6)

(value = [0.7312582738840858, 0.14230146756801187, 0.4738761147449839, 0.33577884268020264, 0.28672100183534543, 0.5702723996544508, 0.1114631361417655, 0.06498506204549281, 0.6026963201960902, 0.15934899135863145  …  0.5100508418962548, 0.027213545126807848, 0.24608087544062318, 0.3805725047594708, 0.36850658016329696, 0.8150473986150026, 0.09005329626760394, 0.6006631159866842, 0.3338958865577528, 0.4993631227903006], time = 0.117126656, bytes = 8020208, gctime = 0.024163857, gcstats = Base.GC_Diff(8020208, 3, 0, 8, 3, 1, 24163857, 1, 0))

In [29]:
e = maximum(f)
jumlah_iterasi = 0
while e > 1e-1
    f = 9.000008-9.0003
  jumlah_iterasi = jumlah_iterasi += 1
    println(jumlah_iterasi)
end